In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
userfile = '/content/drive/My Drive/IS6101project/userInfo.csv'
trainDataTweetFile = '/content/drive/My Drive/IS6101project/FNSC_mapped_users_train.json'
testDataTweetFile = '/content/drive/My Drive/IS6101project/FNSC_mapped_users_test.json'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv(userfile) #.sample(100000, random_state = RSEED)
df.head()

,id,name,location,protected,verified,followers_count,friends_count,favourites_count,statuses_count,created_at,default_profile,default_profile_image
0,84437700,The Good Life Agenda,"San Francisco, CA",False,False,666,1203,131,4072,2009-10-22 22:14:12,False,False
1,26866991,Pete Witte,NaN,False,False,1494,1167,4169,6707,2009-03-26 22:10:15,False,False
2,245228712,Sara Elyse,Cincinnati,False,False,8392,599,38944,15906,2011-01-31 05:27:19,False,False
3,1917731,The Hill,"Washington, DC",False,True,4260775,285,10,939739,2007-03-22 18:15:18,False,False
4,472948434,Chris Wetterich,Cincinnati,False,False,8521,1242,1194,45302,2012-01-24 13:39:43,True,False


In [ ]:
df.columns
ids = df['id'].to_numpy().astype(str)
print(type(ids[0]))

<class 'numpy.str_'>


In [ ]:
labelpath = '/content/drive/ My Drive/IS6101project/userLabel.json'
file = open(labelpath, "r")

contents = file.read()
userLabel = ast.literal_eval(contents)

file.close()

In [ ]:
labels = []
for user in ids:
  labels.append(userLabel[user])

print(len(labels))


96099


In [ ]:
df['labels'] = labels
df.columns

Index(['id', 'name', 'location', 'protected', 'verified', 'followers_count',
       'friends_count', 'favourites_count', 'statuses_count', 'created_at',
       'default_profile', 'default_profile_image', 'labels'],
      dtype='object')

In [ ]:
df.isna().sum()

id                           0
name                         6
location                 22818
protected                    0
verified                     0
followers_count              0
friends_count                0
favourites_count             0
statuses_count               0
created_at                   0
default_profile              0
default_profile_image        0
labels                       0
dtype: int64

In [ ]:
df['labels'].value_counts()

1    48979
0    47120
Name: labels, dtype: int64

In [ ]:
print(df['location'].nunique())
df['location'].unique()[0:10]

30205


array(['San Francisco, CA  ', nan, 'Cincinnati', 'Washington, DC',
       'Cincinnati, OH', 'Kuching, Sarawak', 'Somewhere on Earth. I 🤔 ',
       'USA', 'United States', 'The United States of America'],
      dtype=object)

In [ ]:
print(df['created_at'].nunique())
df['created_at'].unique()[:10]

96086


array(['2009-10-22 22:14:12', '2009-03-26 22:10:15',
       '2011-01-31 05:27:19', '2007-03-22 18:15:18',
       '2012-01-24 13:39:43', '2020-03-16 17:39:23',
       '2010-07-09 14:03:03', '2009-07-12 02:47:40',
       '2020-09-03 15:27:10', '2021-01-12 20:40:20'], dtype=object)

In [ ]:
df['location'] = df['location'].fillna('Missing')

In [ ]:
print(df['location'].nunique())
df['location'].unique()[0:10]

30206


array(['San Francisco, CA  ', 'Missing', 'Cincinnati', 'Washington, DC',
       'Cincinnati, OH', 'Kuching, Sarawak', 'Somewhere on Earth. I 🤔 ',
       'USA', 'United States', 'The United States of America'],
      dtype=object)

In [ ]:
pd.value_counts(df.location)

Missing                  22818
Washington, DC            1215
United States             1175
New York, NY               792
London                     678
                         ...  
Cochin, Kerala               1
Salisbury                    1
Silicon Valley/Hawaii        1
Zapopan, Jal. México.        1
Ho Chi Minh City             1
Name: location, Length: 30206, dtype: int64

In [ ]:
X = df[[ 'protected', 'verified', 'followers_count', 'friends_count', 'favourites_count', 'statuses_count', 'default_profile', 'default_profile_image']] #listed_count is missing 'location', 'created_at', 
len(X)
Y = df['labels'] #.astype(int)
# tfMap = {True: 1, False: 0}
X['protected'] = X['protected'].astype(int)
X['verified'] = X['verified'].astype(int)
X['default_profile'] = X['default_profile'].astype(int)
X['default_profile_image'] = X['default_profile_image'].astype(int)
# X['protected'].value_counts()

scaler = MinMaxScaler()
X[['followers_count', 'friends_count','favourites_count', 'statuses_count']] = scaler.fit_transform(
                                                      X[['followers_count', 'friends_count','favourites_count', 'statuses_count']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [ ]:
X['followers_count'][:5]

0    0.000005
1    0.000011
2    0.000065
3    0.032751
4    0.000065
Name: followers_count, dtype: float64

In [ ]:
cat_cols = list(df.select_dtypes(['object']).columns.values)
print(cat_cols)
X_categorical = df[['created_at']]. \
                  apply(lambda x: x.astype('category').cat.codes)
Xfull = X.join(X_categorical)

['name', 'location', 'created_at', 'labels']


In [ ]:
RSEED = 50
train, test, train_labels, test_labels = train_test_split(Xfull, Y, 
                                                          stratify = labels,
                                                          test_size = 0.2, 
                                                          random_state = RSEED)

In [ ]:
clf = RandomForestClassifier(criterion='gini',
                      n_estimators=100,
                      bootstrap=True,
                      oob_score=True,
                      n_jobs=-1,
                      random_state=RSEED,
                      max_features=9)
clf.fit(train, train_labels)
# clf.score(test, test_labels)
y_pred = clf.predict(test)
accuracy_score(test_labels, y_pred)
print(f"The accuracy of the model is {round(accuracy_score(test_labels,y_pred),3)*100} %")
print(f"The f1-score of the model is {round(f1_score(test_labels,y_pred, pos_label='1'),3)*100} %")
print(f"The precision-score of the model is {round(precision_score(test_labels,y_pred, pos_label='1'),3)*100} %")
print(f"The recall-score of the model is {round(recall_score(test_labels,y_pred, pos_label='1'),3)*100} %")

The accuracy of the model is 58.699999999999996 %
The f1-score of the model is 60.4 %
The precision-score of the model is 59.099999999999994 %
The recall-score of the model is 61.8 %


In [ ]:
train.dtypes

location                 object
protected                 int64
verified                  int64
followers_count           int64
friends_count             int64
favourites_count          int64
statuses_count            int64
created_at               object
default_profile           int64
default_profile_image     int64
dtype: object

In [ ]:
features_to_encode = list(X.select_dtypes(include = ['object']).columns) 
col_trans = make_column_transformer(
                        (OneHotEncoder(handle_unknown='ignore'),features_to_encode),
                        remainder = "passthrough"
                        )


In [ ]:
rf_classifier = RandomForestClassifier(
                      criterion='gini',
                      n_estimators=100,
                      bootstrap=True,
                      oob_score=True,
                      n_jobs=-1,
                      random_state=RSEED,
                      max_features=8)
# pipe = make_pipeline(col_trans, rf_classifier)
# pipe.fit(train, train_labels)
rf_classifier.fit(train, train_labels)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=8,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=50, verbose=0,
                       warm_start=False)

In [ ]:
# y_pred = pipe.predict(test)
y_pred = rf_classifier.predict(test)
accuracy_score(test_labels, y_pred)
print(f"The accuracy of the model is {round(accuracy_score(test_labels,y_pred),3)*100} %")
print(f"The f1-score of the model is {round(f1_score(test_labels,y_pred, pos_label='1'),3)*100} %")

The accuracy of the model is 56.99999999999999 %
The f1-score of the model is 58.5 %


Trial1:    Accuracy = 61.199%
- No minmax scaling of integer features
- ignore mismatch in locations between train and test for one hot encoding
- use raw location and created_at data

Trial2:    Accuracy = 61.1999%
- MinMax scaling for integer features
- same as pts 2,3 as trial 1

Trial3:    Accuracy = 56.99%,  F1-Score = 58.5%
- Removed location and created at features 
- used same setup as above

Trial4: 
The accuracy of the model is 60.099999999999994 %
The f1-score of the model is 61.199999999999996 %
The precision-score of the model is 60.699999999999996 %
The recall-score of the model is 61.7 %
- Used location and created at with different categorical conversion
- used min max scaling

Trial5:    
The accuracy of the model is 59.099999999999994 %
The f1-score of the model is 60.0 %
The precision-score of the model is 59.8 %
The recall-score of the model is 60.3 %
- used only location no created at

Trial6:      
The accuracy of the model is 58.699999999999996 %
The f1-score of the model is 60.4 %
The precision-score of the model is 59.099999999999994 %
The recall-score of the model is 61.8 %
- used only created at and not location

In [ ]:
import json
userLabel = {}
with open(trainDataTweetFile, 'r') as f:
    for x in f:
        x = json.loads(x)
        # print(x['user_ids'])
        for uid in x['user_ids']:
          if uid in userLabel:
            continue
          else:
            if uid in ids:
              userLabel[uid] = x['label']
            else:
              continue


In [ ]:
len(userLabel)

88071

In [ ]:
with open(testDataTweetFile, 'r') as f:
    for x in f:
        x = json.loads(x)
        for uid in x['user_ids']:
          if uid in userLabel:
            continue
          else:
            if uid in ids:
              userLabel[uid] = x['label']
            else:
              continue

In [ ]:
len(userLabel)

96099